In [ ]:
import os
import sys

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Dataset preprocessing

In [ ]:
vdjdb_raw = pd.read_csv('data/raw/vdjdb.tsv', sep='\t')
iedb_raw = pd.read_csv('data/raw/iedb_mhc1.csv')